In [1]:
import keras 
from keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

train_data.shape, train_targets.shape

((404, 13), (404,))

In [3]:
HIDDEN_SIZES = [ 20, 20 ]

In [4]:
from sklearn.metrics import mean_squared_error
def cross_val_score(X, y, build_model, cv=10):
    X = np.array(X)
    y = np.array(y)
    part = len(X) // cv
    results = np.empty(cv)
    for i in range(cv):
        train_X = np.concatenate(
            (X[:i*part],
            X[((i+1)*part):])
        )
        train_y = np.concatenate(
            (y[:i*part],
            y[(i+1)*part:])
        )
        test_X = X[i*part:(i+1)*part]
        test_y = y[i*part:(i+1)*part]
        
        model = build_model()
        
        model.fit(train_X, train_y, epochs=40, batch_size=16, verbose=0)
        predicted = model.predict(test_X)
        
        results[i] = mean_squared_error(test_y, predicted)
        
        print('score %d: %f' % (i + 1, results[i]))
        
    return results.mean()

# Sequential API

In [5]:
def build_seq_model(input_shape=13):
    model = keras.models.Sequential()
    for index, size in enumerate(HIDDEN_SIZES):
        if index == 0:
            model.add(layers.Dense(size, activation='relu', input_shape=(input_shape,)))
        else:
            model.add(layers.Dense(size, activation='relu'))
    model.add(layers.Dense(1))
    model.compile('rmsprop', loss='mse')
    
    return model

In [23]:
model = build_seq_model()
model.fit(train_data, train_targets, epochs=120, batch_size=32, verbose=0)

In [7]:
%%time
cross_val_score(train_data, train_targets, build_seq_model)

score 1: 108.832809
score 2: 9.489520
score 3: 41.941970
score 4: 43.703529
score 5: 37.561281
score 6: 23.920821
score 7: 16.590272
score 8: 68.630472
score 9: 86.361597
score 10: 60.280829
Wall time: 13.2 s


49.731309860103046

In [8]:
# linear predictions

from sklearn import model_selection
from sklearn.linear_model import LinearRegression
-model_selection.cross_val_score(LinearRegression(), train_data, train_targets, cv=10, scoring='neg_mean_squared_error').mean()

25.075378478863474

# Functional API

In [21]:
from keras.layers import Input, Dense
from keras.models import Model

def build_func_model(input_shape=13):
    inputs = x = Input(shape=(13,))
    
    for size in HIDDEN_SIZES:
        x = Dense(size, activation='relu')(x)
        
    predictions = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile('rmsprop', loss='mse')
    
    return model

In [10]:
%%time
cross_val_score(train_data, train_targets, build_func_model)

score 1: 76.499378
score 2: 57.663296
score 3: 52.520988
score 4: 41.615008
score 5: 66.624906
score 6: 84.218448
score 7: 20.797712
score 8: 65.058590
score 9: 84.092903
score 10: 33.364580
Wall time: 15.6 s


58.245580978760415

# Subclassing API

In [11]:
class SubModel(Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(20, activation='relu')
        self.dense2 = Dense(20, activation='relu')
        self.predictions =  Dense(1)
        self.compile('rmsprop', loss='mse')
        
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        
        return self.predictions(x)

def build_subclassing_model():
    return SubModel()

In [12]:
%%time
cross_val_score(train_data, train_targets, build_subclassing_model)

score 1: 49.905433
score 2: 27.165728
score 3: 45.887543
score 4: 64.571509
score 5: 40.305240
score 6: 23.996428
score 7: 20.641424
score 8: 62.366696
score 9: 88.188319
score 10: 36.851490
Wall time: 17.9 s


45.98798100920051

In [60]:
class SubModel(Model):
    def __init__(self):
        super().__init__()
        self.denses = [ Dense(size, activation='relu') for size in HIDDEN_SIZES ]
        self.denses.append(Dense(1))
        
        for index, dense in enumerate(self.denses):
            setattr(self, '__dense%d__' % (index + 1), dense)
        
        self.compile('rmsprop', loss='mse')
        
    def call(self, x):
        for dense in self.denses:
            x = dense(x)
        return x

def build_subclassing_model():
    return SubModel()

model = build_subclassing_model()
model.fit(train_data, train_targets, batch_size=16, epochs=10)

Epoch 1/10
404/404 [==============================] - 1s 2ms/step - loss: 1180.9942
Epoch 2/10
404/404 [==============================] - 0s 123us/step - loss: 94.1677
Epoch 3/10
404/404 [==============================] - 0s 128us/step - loss: 90.8009
Epoch 4/10
404/404 [==============================] - 0s 160us/step - loss: 86.3676: 0s - loss: 86.897
Epoch 5/10
404/404 [==============================] - 0s 170us/step - loss: 76.9403
Epoch 6/10
404/404 [==============================] - 0s 279us/step - loss: 78.9879
Epoch 7/10
404/404 [==============================] - 0s 257us/step - loss: 68.9439
Epoch 8/10
404/404 [==============================] - 0s 185us/step - loss: 67.4442
Epoch 9/10
404/404 [==============================] - 0s 170us/step - loss: 64.3151
Epoch 10/10
404/404 [==============================] - 0s 207us/step - loss: 61.5531
